In [6]:
import pandas as pd
import numpy as np
import gzip
import os
import glob

## Data cleaning

### Looping over every zipped state data and read as csv

In [30]:

zipfiles = "/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_shipment/*.gz"

filelist = glob.glob(zipfiles)

select_data = []

for gzfile in filelist:
    raw = pd.read_csv(gzfile, iterator= True, chunksize = 10000, compression='gzip', error_bad_lines=False)


    for df in raw:

        df["TRANSACTION_DATE1"] = pd.to_datetime(df["TRANSACTION_DATE"], format='%m%d%Y')

        df["year"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).year

        group_df = df.groupby(["BUYER_STATE", "BUYER_COUNTY", "year"], as_index=False).sum("MME")

        group_df = group_df[["BUYER_COUNTY", "BUYER_STATE", "year", "MME"]]

        select_data.append(group_df)



whole_data = pd.concat(select_data)


# Test if the data is correctly loaded

print(len(whole_data))

whole_data["BUYER_STATE"].value_counts()

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


1965604


FL    514367
GA    497486
AL    207252
OK    170215
LA    166679
MS    138045
SC    130671
OR     80768
NM     32845
MT     27276
Name: BUYER_STATE, dtype: int64

### Store data in parquet form

In [31]:
whole_data.to_parquet('/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/opioid_whole.parquet', engine='fastparquet')